Working jupyter to create function to create a list of aminoacids likely to be used for mutation

Function finished

In [2]:
path_pdb = './data/pdbs'
path_pqr=  './data/pqrs'
pdb_file = 'AF-C0SP86-F1.pdb'
pqr_file = 'P39846.pqr'
output_path = './data/test'
name = pqr_file.split('.')[0]
from function import functional_aa
from function import free_aa
from helper_function import pdb2AA

functional_aas = functional_aa(path_pdb, pdb_file, output_path, df =False)

free_aas = free_aa(path_pdb, pdb_file, functional_aas)

Pqr file already exists


c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:71: RuntimeWarning: invalid value encountered in cast
  distance[:,0] = distance[:,0].astype('int')


Salt_bridge finished


c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:89: RuntimeWarning: invalid value encountered in arccos
  theta = np.arccos((d_DH[n,1,1]**2 + d_HA[n,1:]**2 - d_DA[1:,1:][n,:]**2)/(2*d_DH[n,1,1]*d_HA[1:,1:][n,:]))


H_bond finished
VdW_interaction finished


c:\Users\marik\OneDrive - bwedu\Uni HD\FS 4\Bioinfo\topic04_02\helper_function.py:106: RuntimeWarning: invalid value encountered in cast
  array[:,0] = array[:,0].astype('int')


In [3]:
from helper_function import pdb2AA
test = pdb2AA(path_pdb, pdb_file)
print(test)

['M', 'S', 'W', 'L', 'H', 'K', 'F', 'F', 'D', 'L', 'F', 'L', 'G', 'E', 'S', 'E', 'E', 'D', 'A', 'E', 'R', 'E', 'T', 'K', 'P', 'A', 'Q', 'I', 'P', 'Q', 'Q', 'Q', 'E', 'V', 'H', 'H', 'P', 'E', 'G', 'Q', 'L', 'K', 'R', 'L', 'E', 'D', 'P', 'K', 'I', 'Y', 'Y', 'E', 'Y', 'P', 'K', 'G', 'K', 'F', 'R', 'F', 'P', 'V', 'V', 'P', 'D', 'G', 'Y', 'K', 'N', 'H', 'D', 'L', 'R', 'R', 'R', 'R', 'T', 'P', 'S', 'D', 'E', 'P', 'K', 'S', 'A', 'P', 'R', 'P', 'S', 'A', 'A', 'P', 'Y', 'R', 'E', 'R', 'P', 'R', 'N', 'E', 'E', 'E', 'Q', 'H', 'T', 'Y', 'Q', 'A', 'A', 'E', 'P', 'A', 'K', 'K', 'P', 'F', 'K', 'P', 'T', 'N', 'I', 'P', 'S', 'P', 'V', 'Y', 'G', 'F', 'N', 'Q', 'K', 'P', 'S', 'V', 'K', 'K', 'D', 'V', 'P', 'K', 'K', 'P', 'S', 'E', 'T', 'L', 'N', 'E', 'P', 'D', 'K', 'S', 'V', 'K', 'E', 'K', 'V', 'T', 'L', 'L', 'S', 'E', 'E', 'I', 'E', 'R', 'E', 'R', 'G', 'Y', 'P', 'A', 'S', 'D', 'T', 'Q', 'A', 'H', 'S', 'K', 'I', 'E', 'S', 'P', 'F', 'F', 'P', 'D', 'T', 'Q', 'F', 'E', 'K', 'Q', 'P', 'S', 'G', 'V', 'L', 'N',

In [7]:
test = [1,2,5,6]
print(len(test))

4


In [50]:
def prot_mut(path, file_name, functional_aa, free_aa):
    
    import os
    from helper_function import pdb2AA
    
    
    # top 20 features that correlate with melt point (positive and negative), based on prokaryotes323 columns, cutoff at +/- 0.32
    pos_corr = ['AR', 'VR', 'LR', 'LP', 'MR', 'FR', 'WR', 'NR', 'YR', 
                'YP', 'ER', 'EP', 'RH', 'RC', 'RP', 'RG', 'PG', 'Rhelix', 
                'R', 'P', 'ARmotif', 'EAmotif'] 
    neg_corr = ['IQ', 'MQ', 'MT', 'FQ', 'WQ', 'NQ', 'QS', 'QT', 'QD', 
                'QH', 'QK', 'QC', 'ST', 'SH', 'SC', 'TD', 'TH', 'TC', 
                'Qhelix', 'Q', 'T', 'PolarAA']
    
    #extract aminoacid list
    aa_list = pdb2AA(path, file_name)
            
    return 

Get list of pos/neg correlating features for Meltpoint

In [6]:
import pandas as pd
import os
pro_df: pd.DataFrame = pd.read_csv(os.path.join('./data', 'prokaryotes_323columns.csv'))
pro_df = pro_df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])

C:\Users\marik\AppData\Local\Temp\ipykernel_5112\768696474.py:3: DtypeWarning: Columns (8,21,22,23,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  pro_df: pd.DataFrame = pd.read_csv(os.path.join('./data', 'prokaryotes_323columns.csv'))


In [16]:
import numpy
numpy.version.version


'1.26.4'

In [39]:
pro_corr = pro_df.corr(numeric_only=True)['meltPoint']
pos_corr = pro_corr[pro_corr > 0.32]
neg_corr = pro_corr[pro_corr < -0.32]

In [40]:
pos_corr = list(pro_corr.index)
#print(pos_corr[3:])
neg_corr = list(neg_corr.index)
print(neg_corr)

['IQ', 'MQ', 'MT', 'FQ', 'WQ', 'NQ', 'QS', 'QT', 'QD', 'QH', 'QK', 'QC', 'ST', 'SH', 'SC', 'TD', 'TH', 'TC', 'Qhelix', 'Q', 'T', 'PolarAA']


Needs to be mutated

In [14]:
print(pro_corr[pro_corr < -0.32])

IQ        -0.429683
MQ        -0.507329
MT        -0.359458
FQ        -0.423630
WQ        -0.420057
NQ        -0.435562
QS        -0.502697
QT        -0.529350
QD        -0.466556
QH        -0.455041
QK        -0.418720
QC        -0.493738
ST        -0.396185
SH        -0.327684
SC        -0.321686
TD        -0.344690
TH        -0.346921
TC        -0.364763
Qhelix    -0.378950
Q         -0.469765
T         -0.321208
PolarAA   -0.406035
Name: meltPoint, dtype: float64
